# Día 3: Manejo de Errores y Excepciones

## Descripción General

El manejo de errores es una parte fundamental de la programación profesional. En lugar de permitir que tu programa se detenga abruptamente cuando algo sale mal, Python te proporciona un sistema robusto de excepciones que te permite anticipar, capturar y responder a errores de manera elegante.

En este notebook aprenderás a usar `try/except` para capturar errores, crear excepciones personalizadas que comuniquen problemas específicos de tu dominio, y propagar errores de manera apropiada a través de tu código. Estas habilidades son esenciales para construir aplicaciones confiables y fáciles de depurar.

## Objetivos de Aprendizaje

Al finalizar este notebook, serás capaz de:

1. Usar bloques `try/except/else/finally` para manejar excepciones de manera efectiva
2. Capturar excepciones específicas y múltiples tipos de excepciones
3. Crear excepciones personalizadas que representen errores específicos de tu dominio
4. Propagar excepciones apropiadamente usando `raise` y re-lanzamiento
5. Aplicar mejores prácticas para el manejo de errores en código de producción

## 1. Fundamentos de Try/Except

### El Problema que Resuelve

Cuando escribes código, muchas cosas pueden salir mal: un archivo puede no existir, un usuario puede ingresar datos inválidos, una conexión de red puede fallar. Sin manejo de errores, tu programa simplemente se detendría con un mensaje de error críptico.

### Visualización del Problema

```
SIN MANEJO DE ERRORES:
Usuario ingresa "abc" → int("abc") → ValueError → PROGRAMA SE DETIENE ❌

CON MANEJO DE ERRORES:
Usuario ingresa "abc" → try: int("abc") → except ValueError → Mensaje amigable → PROGRAMA CONTINÚA ✓
```

### Ejemplo: Código sin Manejo de Errores (MAL)

In [ ]:
# BAD: No error handling - program crashes
def calculate_average_bad(numbers):
    """
    Calculate average without error handling.
    
    :param numbers: List of numbers
    :type numbers: list
    :return: Average value
    :rtype: float
    """
    return sum(numbers) / len(numbers)

# This will crash if list is empty
# result = calculate_average_bad([])  # ZeroDivisionError!

### Ejemplo: Código con Manejo de Errores (BIEN)

In [ ]:
# GOOD: Proper error handling
def calculate_average_good(numbers):
    """
    Calculate average with error handling.
    
    :param numbers: List of numbers
    :type numbers: list
    :return: Average value or None if list is empty
    :rtype: float | None
    """
    try:
        return sum(numbers) / len(numbers)
    except ZeroDivisionError:
        print("Error: Cannot calculate average of empty list")
        return None
    except TypeError:
        print("Error: List contains non-numeric values")
        return None

# Now it handles errors gracefully
print(calculate_average_good([]))  # None
print(calculate_average_good([1, 2, 3, 4, 5]))  # 3.0

### Aprendizaje Clave

El bloque `try/except` te permite anticipar errores y responder de manera controlada. Siempre captura excepciones específicas (como `ZeroDivisionError`) en lugar de usar un `except` genérico, para evitar ocultar errores inesperados.

**Referencia oficial:** [Python Tutorial - Errors and Exceptions](https://docs.python.org/tutorial/errors.html)

## 2. Bloques Else y Finally

### El Problema que Resuelve

A veces necesitas ejecutar código solo si NO ocurrió una excepción (`else`), o necesitas ejecutar código de limpieza sin importar qué pase (`finally`). Estos bloques opcionales te dan control fino sobre el flujo de ejecución.

### Estructura Completa de Try/Except

In [ ]:
def read_file_safely(filename):
    """
    Read file with complete error handling.
    
    :param filename: Path to file
    :type filename: str
    :return: File contents or None
    :rtype: str | None
    """
    file_handle = None
    try:
        file_handle = open(filename, 'r')
        content = file_handle.read()
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found")
        return None
    except PermissionError:
        print(f"Error: No permission to read '{filename}'")
        return None
    else:
        # Only executes if NO exception occurred
        print(f"Successfully read {len(content)} characters")
        return content
    finally:
        # ALWAYS executes, even if exception occurred
        if file_handle:
            file_handle.close()
            print("File closed")

# Test with non-existent file
result = read_file_safely("nonexistent.txt")

### Aprendizaje Clave

- `else`: Se ejecuta solo si NO hubo excepciones en el bloque `try`
- `finally`: Se ejecuta SIEMPRE, sin importar si hubo excepciones o no. Ideal para limpieza de recursos (cerrar archivos, conexiones, etc.)

**Referencia oficial:** [Built-in Exceptions](https://docs.python.org/3/library/exceptions.html)

### Pregunta de Comprensión

¿Cuál es la diferencia entre poner código después del bloque `try/except` versus ponerlo en el bloque `else`?

## 3. Excepciones Personalizadas

### El Problema que Resuelve

Las excepciones built-in de Python (`ValueError`, `TypeError`, etc.) son genéricas. Cuando construyes una aplicación, necesitas excepciones que representen errores específicos de tu dominio, como `InvalidEmailError` o `InsufficientFundsError`.

### Ejemplo: Excepciones Genéricas (MAL)

In [ ]:
# BAD: Using generic exceptions
def withdraw_money_bad(balance, amount):
    """
    Withdraw money using generic exceptions.
    
    :param balance: Current balance
    :type balance: float
    :param amount: Amount to withdraw
    :type amount: float
    :return: New balance
    :rtype: float
    """
    if amount > balance:
        raise ValueError("Not enough money")  # Too generic!
    return balance - amount

### Ejemplo: Excepciones Personalizadas (BIEN)

In [ ]:
# GOOD: Custom exceptions with clear meaning
class InsufficientFundsError(Exception):
    """
    Raised when attempting to withdraw more money than available.
    
    :param balance: Current balance
    :type balance: float
    :param amount: Attempted withdrawal amount
    :type amount: float
    """
    def __init__(self, balance, amount):
        self.balance = balance
        self.amount = amount
        self.shortfall = amount - balance
        super().__init__(
            f"Insufficient funds: balance={balance}, "
            f"requested={amount}, shortfall={self.shortfall}"
        )

class NegativeAmountError(Exception):
    """
    Raised when attempting to withdraw negative amount.
    """
    pass

def withdraw_money_good(balance, amount):
    """
    Withdraw money with custom exceptions.
    
    :param balance: Current balance
    :type balance: float
    :param amount: Amount to withdraw
    :type amount: float
    :return: New balance
    :rtype: float
    :raises InsufficientFundsError: If balance is insufficient
    :raises NegativeAmountError: If amount is negative
    """
    if amount < 0:
        raise NegativeAmountError("Cannot withdraw negative amount")
    if amount > balance:
        raise InsufficientFundsError(balance, amount)
    return balance - amount

# Now we can catch specific errors
try:
    new_balance = withdraw_money_good(100.0, 150.0)
except InsufficientFundsError as e:
    print(f"Transaction failed: {e}")
    print(f"You need ${e.shortfall:.2f} more")

### Aprendizaje Clave

Las excepciones personalizadas deben heredar de `Exception` (no de `BaseException`). Dales nombres descriptivos que terminen en `Error` y proporciona información útil en el mensaje. Esto hace que tu código sea más expresivo y fácil de depurar.

**Referencia oficial:** [PEP 352 - Required Superclass for Exceptions](https://www.python.org/dev/peps/pep-0352/)

## 4. Propagación de Excepciones

### El Problema que Resuelve

A veces necesitas capturar una excepción, hacer algo (como logging), y luego re-lanzarla para que el código que llamó tu función también pueda manejarla. Otras veces necesitas transformar una excepción en otra más apropiada para tu nivel de abstracción.

### Ejemplo: Re-lanzar Excepciones

In [ ]:
def process_user_data(user_id):
    """
    Process user data with exception re-raising.
    
    :param user_id: User identifier
    :type user_id: int
    :raises ValueError: If user_id is invalid
    """
    try:
        # Simulate database operation
        if user_id < 0:
            raise ValueError("User ID must be positive")
        print(f"Processing user {user_id}")
    except ValueError as e:
        # Log the error
        print(f"[LOG] Error processing user {user_id}: {e}")
        # Re-raise the same exception
        raise  # No argument = re-raise the current exception

# The caller can also handle it
try:
    process_user_data(-5)
except ValueError:
    print("Caller: Caught the re-raised exception")

### Ejemplo: Encadenar Excepciones

In [ ]:
class DataProcessingError(Exception):
    """
    High-level exception for data processing errors.
    """
    pass

def parse_config_file(filename):
    """
    Parse configuration file with exception chaining.
    
    :param filename: Path to config file
    :type filename: str
    :return: Parsed configuration
    :rtype: dict
    :raises DataProcessingError: If parsing fails
    """
    try:
        with open(filename, 'r') as f:
            # Simulate parsing
            content = f.read()
            if not content:
                raise ValueError("Empty config file")
            return {"config": "data"}
    except (FileNotFoundError, ValueError) as e:
        # Transform low-level exception to high-level one
        # 'from e' preserves the original exception
        raise DataProcessingError(
            f"Failed to parse config from {filename}"
        ) from e

# The caller sees the high-level exception
try:
    config = parse_config_file("missing.conf")
except DataProcessingError as e:
    print(f"Error: {e}")
    print(f"Original cause: {e.__cause__}")

### Aprendizaje Clave

- Usa `raise` sin argumentos para re-lanzar la excepción actual
- Usa `raise NewException() from original_exception` para encadenar excepciones y preservar el contexto
- Transforma excepciones de bajo nivel en excepciones de alto nivel apropiadas para tu API

**Referencia oficial:** [Python Tutorial - Exception Chaining](https://docs.python.org/tutorial/errors.html#exception-chaining)

## 5. Mejores Prácticas

### Principios Clave del Manejo de Errores

In [ ]:
# 1. Be specific: Catch specific exceptions
try:
    value = int(input("Enter a number: "))
except ValueError:  # GOOD: Specific
    print("Invalid number")

# 2. Don't catch everything
# try:
#     risky_operation()
# except:  # BAD: Catches everything, even KeyboardInterrupt!
#     pass

# 3. Fail fast: Validate inputs early
def calculate_discount(price, discount_percent):
    """
    Calculate discounted price with early validation.
    
    :param price: Original price
    :type price: float
    :param discount_percent: Discount percentage (0-100)
    :type discount_percent: float
    :return: Discounted price
    :rtype: float
    :raises ValueError: If inputs are invalid
    """
    # Validate early
    if price < 0:
        raise ValueError("Price cannot be negative")
    if not 0 <= discount_percent <= 100:
        raise ValueError("Discount must be between 0 and 100")
    
    return price * (1 - discount_percent / 100)

# 4. Provide context in error messages
class InvalidConfigError(Exception):
    """
    Exception with detailed context.
    """
    def __init__(self, field, value, reason):
        self.field = field
        self.value = value
        self.reason = reason
        super().__init__(
            f"Invalid config field '{field}' with value '{value}': {reason}"
        )

print("Best practices demonstrated!")

### Aprendizaje Clave

1. **Sé específico**: Captura excepciones específicas, no genéricas
2. **Falla rápido**: Valida inputs al inicio de la función
3. **Proporciona contexto**: Incluye información útil en los mensajes de error
4. **No silencies errores**: Evita `except: pass` que oculta problemas
5. **Documenta excepciones**: Usa `:raises:` en docstrings para documentar qué excepciones puede lanzar tu función

## Ejercicios Prácticos

### Ejercicio 1: Validación de Email

Crea una función `validate_email(email)` que:
- Lance `InvalidEmailError` si el email no contiene '@'
- Lance `InvalidEmailError` si el email no contiene '.'
- Lance `InvalidEmailError` si el email está vacío
- Retorne `True` si el email es válido

Crea la excepción personalizada `InvalidEmailError` con un mensaje descriptivo.

In [ ]:
# TODO: Implement InvalidEmailError and validate_email

# Test your implementation
# try:
#     validate_email("user@example.com")  # Should work
#     validate_email("invalid-email")  # Should raise InvalidEmailError
# except InvalidEmailError as e:
#     print(f"Caught: {e}")

### Ejercicio 2: Lectura Segura de Archivos

Crea una función `read_json_file(filename)` que:
- Use try/except/else/finally apropiadamente
- Maneje `FileNotFoundError` y `json.JSONDecodeError`
- Retorne el contenido parseado o `None` si hay error
- Imprima un mensaje en el bloque `finally` indicando que terminó el intento

In [ ]:
import json

# TODO: Implement read_json_file

# Test your implementation
# result = read_json_file("test.json")

### Ejercicio 3: Sistema de Transacciones Bancarias

Crea un sistema de transacciones con:
- Excepción `InsufficientFundsError` con atributos `balance` y `amount`
- Excepción `InvalidTransactionError` para montos negativos o cero
- Función `transfer_money(from_account, to_account, amount)` que valide y lance las excepciones apropiadas
- Función `process_transaction()` que capture y maneje estas excepciones, mostrando mensajes apropiados

In [ ]:
# TODO: Implement the banking transaction system

# Test your implementation
# accounts = {"Alice": 1000, "Bob": 500}
# try:
#     transfer_money(accounts, "Alice", "Bob", 1500)
# except InsufficientFundsError as e:
#     print(f"Transaction failed: {e}")

## Resumen

En este notebook has aprendido:

1. **Try/Except básico**: Capturar excepciones específicas para manejar errores de manera controlada
2. **Else y Finally**: Usar `else` para código que solo se ejecuta si no hay errores, y `finally` para limpieza que siempre debe ocurrir
3. **Excepciones personalizadas**: Crear excepciones que representen errores específicos de tu dominio, heredando de `Exception`
4. **Propagación de errores**: Re-lanzar excepciones con `raise` y encadenarlas con `from` para preservar contexto
5. **Mejores prácticas**: Ser específico, fallar rápido, proporcionar contexto, y documentar excepciones

El manejo apropiado de errores es lo que separa código amateur de código profesional. Te permite construir aplicaciones robustas que fallan de manera predecible y proporcionan información útil para debugging.

## Preguntas de Autoevaluación

### 1. ¿Cuál es la diferencia entre el bloque `else` y simplemente poner código después del `try/except`?

**Respuesta:** El bloque `else` solo se ejecuta si NO ocurrió ninguna excepción en el bloque `try`. El código después del `try/except` se ejecuta siempre, independientemente de si hubo una excepción o no. Usar `else` hace que la intención del código sea más clara.

### 2. ¿Por qué es mejor capturar excepciones específicas en lugar de usar `except:` genérico?

**Respuesta:** Capturar excepciones específicas evita ocultar errores inesperados. Un `except:` genérico captura TODO, incluyendo `KeyboardInterrupt` y `SystemExit`, lo que puede hacer que tu programa sea imposible de detener. Además, excepciones específicas te permiten manejar diferentes errores de manera diferente.

### 3. ¿Cuándo deberías crear una excepción personalizada en lugar de usar una built-in?

**Respuesta:** Crea excepciones personalizadas cuando necesites representar errores específicos de tu dominio que no están bien representados por las excepciones built-in. Por ejemplo, `InsufficientFundsError` es más descriptivo que `ValueError` en un sistema bancario. Las excepciones personalizadas también pueden incluir atributos adicionales con contexto útil.

### 4. ¿Qué hace `raise` sin argumentos dentro de un bloque `except`?

**Respuesta:** `raise` sin argumentos re-lanza la excepción actual que está siendo manejada. Esto es útil cuando quieres hacer algo (como logging) pero también quieres que el código que llamó tu función pueda manejar la excepción.

### 5. ¿Cuál es el propósito de `raise NewException() from original_exception`?

**Respuesta:** Esta sintaxis encadena excepciones, preservando la excepción original como causa de la nueva. Esto es útil cuando transformas excepciones de bajo nivel en excepciones de alto nivel más apropiadas para tu API, pero quieres mantener el contexto completo para debugging. La excepción original queda accesible en el atributo `__cause__`.

### 6. ¿Por qué es importante validar inputs al inicio de una función ("fail fast")?

**Respuesta:** Validar inputs temprano evita que datos inválidos se propaguen a través de tu código, causando errores confusos más adelante. Es más fácil debuggear cuando el error ocurre cerca de donde se introdujo el problema. Además, proporciona mensajes de error más claros al usuario.

### 7. ¿Qué información debería incluir una buena excepción personalizada?

**Respuesta:** Una buena excepción personalizada debe incluir: (1) Un nombre descriptivo que termine en `Error`, (2) Un mensaje claro explicando qué salió mal, (3) Atributos con contexto relevante (como valores que causaron el error), (4) Documentación en el docstring explicando cuándo se lanza.

## Recursos y Referencias Oficiales

### Documentación Oficial
- **[Python Tutorial - Errors and Exceptions](https://docs.python.org/tutorial/errors.html)**: Tutorial oficial sobre manejo de errores y excepciones en Python
- **[Built-in Exceptions](https://docs.python.org/3/library/exceptions.html)**: Referencia completa de todas las excepciones built-in de Python y su jerarquía

### Estándares/PEPs
- **[PEP 352 - Required Superclass for Exceptions](https://www.python.org/dev/peps/pep-0352/)**: Define la jerarquía de excepciones y por qué todas deben heredar de `BaseException`

### Mejores Prácticas
- **[Real Python - Python Exceptions](https://realpython.com/python-exceptions/)**: Guía completa sobre excepciones con ejemplos prácticos
- **[Python Exception Handling Best Practices](https://docs.python-guide.org/writing/gotchas/#mutable-default-arguments)**: Patrones y anti-patrones comunes en manejo de excepciones

### Notas Importantes
- Todos los enlaces están actualizados a partir de 2025
- Se recomienda revisar la documentación oficial regularmente
- La jerarquía de excepciones puede variar ligeramente entre versiones de Python